### Задание
1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)
2. скачиваем готовый новостной датасет
скачиваем из
https://github.com/natasha/corus/blob/master/README.md
любой новостной датасет

    реализовать метод поиска ближайших статей
    (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу)
    визуально оценить качество

In [2]:
!pip install transformers

In [10]:
!pip install sacremoses

     -------------------------------------- 880.6/880.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=0291cded8a05e25b766a25a00a4ef0eea93b962eebb14bfac98d86f4c6f91148
  Stored in directory: c:\users\uliana\appdata\local\pip\cache\wheels\82\ab\9b\c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [24]:
!pip install sentence_transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached torchvision-0.13.1-cp38-cp38-win_amd64.whl (1.1 MB)


1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)

In [36]:
import io
import re
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, util
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

In [4]:
path_to_translate = "rus-eng/rus.txt"

In [5]:
f = open(path_to_translate, encoding="utf8")
for line in f:
    print(line)
    break

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)



In [6]:
def preprocess_sentence(w):
    w = w.lower().strip()

    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
    w = w.strip()

    return w

In [7]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [8]:
en, ru = create_dataset(path_to_translate, None)
print(en[1])
print(ru[1])

go .
иди .


In [11]:
mname = "facebook/wmt19-ru-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

Downloading:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

In [12]:
for i in range(10):
    input = ru[i]
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}, real: {en[i]}')

C:\Users\Uliana\anaconda3\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ru: марш !, en: march!, real: go .


C:\Users\Uliana\anaconda3\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ru: иди ., en: Go., real: go .
ru: идите ., en: Go., real: go .
ru: здравствуйте ., en: Hello., real: hi .
ru: привет !, en: Hello!, real: hi .
ru: хай ., en: Hai., real: hi .
ru: здрасте ., en: , real: hi .
ru: здоро во !, en: Good morning and welcome!, real: hi .
ru: приветик !, en: Hello!, real: hi .
ru: беги !, en: Run!, real: run !


In [13]:
inputs = ['Морозы сковали землю', 'Замерзли реки и озера', 'Везде лежит белый пушистый снег', ' Дети рады зиме', 
          'По свежему снегу приятно бежать на лыжах', 'Матвей и Лера играют в снежки', 'Андрей и Саша лепят снежную бабу', 
          'Только животным тяжело в зимнюю стужу', 'Птицы летят ближе к жилью', 'Ребята, помогите зимой нашим маленьким друзьям', 
          'Сделайте для птиц кормушки']
for input in inputs:
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}')

ru: Морозы сковали землю, en: Frost shackled the earth
ru: Замерзли реки и озера, en: Rivers and lakes frozen
ru: Везде лежит белый пушистый снег, en: White, fluffy snow lies everywhere
ru:  Дети рады зиме, en: Children are happy in winter
ru: По свежему снегу приятно бежать на лыжах, en: On fresh snow it is pleasant to run on skis
ru: Матвей и Лера играют в снежки, en: Matthew and Lera Play Snowball
ru: Андрей и Саша лепят снежную бабу, en: Andrei and Sascha sculpt a snow babe
ru: Только животным тяжело в зимнюю стужу, en: Only animals find it hard in the winter cold
ru: Птицы летят ближе к жилью, en: Birds fly closer to home
ru: Ребята, помогите зимой нашим маленьким друзьям, en: Guys, help our little friends in winter
ru: Сделайте для птиц кормушки, en: Make feeders for birds


Вполне неплохо справился с переводом, даже с сугубр русскими литературными выражениями

2. скачиваем готовый новостной датасет скачиваем из https://github.com/natasha/corus/blob/master/README.md любой новостной датасет

In [14]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [18]:
df = pd.DataFrame(records)

In [19]:
df.head()

,0,1,2,3,4,5
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды,None
1,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,None
2,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,None
3,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,None
4,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,None


In [20]:
df.shape

(739350, 6)

реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу) визуально оценить качество

In [26]:
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/356 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [66]:
sentences = df[2][:2000]

In [67]:
embeddings = model.encode(sentences, convert_to_tensor=True)

In [68]:
def get_similar(embeddings, sentence, n):
    """Get <n> similar embeddings for <sentence> and given embeddings"""
    
    assert n > 0, "Bad n"
    
    # Compute embeddings
    s_emb =  model.encode([sentence], convert_to_tensor=True)

    # Compute cosine-similarities for each sentence
    cosine_scores = util.cos_sim(s_emb, embeddings)
    
    # Compute max n scores indexes
    max_scores_idx = np.argpartition(cosine_scores[0], -n)[-n:]
    
    return cosine_scores, max_scores_idx

In [69]:
rnd = np.random.randint(1000, df.shape[0])
sentence = df.iloc[rnd, 2]

sentence

'США приняли решение усилить охрану посольства в Судане после того, как президенту этой страны Омару аль-Баширу были предъявлены обвинения в геноциде, сообщает агентство Reuters. Представитель Госдепартамента США Шон Маккормак заявил, что "надлежащие меры по обеспечению безопасности" будут приняты для защиты не только персонала посольства, но и вообще всех находящихся в Судане американцев. Никаких подробностей он, однако, не сообщил. Как сообщает агентство France Presse президент США Джордж Буш со своей стороны выразил "серьезную озабоченность" ситуацией в суданском Дарфуре и заявил, что США будут содействовать усилению роли миротворцев ООН и Африканского союза в Судане. Напомним, ранее в понедельник главный прокурор Международного уголовного суда в Гааге обвинил аль-Башира в геноциде и преступлениях против человечности, потребовав его ареста. По словам Маккормака, эти обвинения могут стать причиной роста напряженности и антиамериканских настроений в Судане. Конфликт между властями Суд

In [75]:
cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 3)

In [76]:
cosine_scores[0][max_scores_idx]

tensor([0.1836, 0.2021, 0.2319])

In [77]:
for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

В Челябинской области электрослесарь Василий Борисенко сжег заживо в автомобиле свою жену и мужа ее сестры. Преступление он объяснил ревностью, сообщается на сайте Челябинского областного суда. Суд установил, что в ночь на 3 января в поселке Кленовка обвиняемый праздновал наступление Нового года в компании жены и семьи ее сестры. Затем сестра жены сообщила Борисенко, что та изменила ему с ее мужем. Тогда он избил жену, затем доехал до дома мужа ее сестры, избил его до бессознательного состояния и бросил в багажник автомобиля. Вернувшись домой, Борисенко несколько раз ударил супругу и также поместил ее в багажник. Он отвез заложников в лес, где поджег транспортное средство, используя зажигалку и рекламные буклеты. Находившиеся внутри мужчина и женщина скончались от отравления угарным газом. Спустя два дня после поджога Борисенко сам пришел в полицию и рассказал об убийстве. Он частично признал вину, оправдав преступление чувством ревности. Его признали виновным в двойном убийстве (пункт

In [78]:
rnd = np.random.randint(1000, df.shape[0])
sentence = df.iloc[rnd, 4]

print(sentence)

cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 5)
print("\n")
print(cosine_scores[0][max_scores_idx])
print("\n")

for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

Все


tensor([0.1729, 0.1829, 0.1836, 0.2021, 0.2319])


Бывшему главе Ирбейского района Красноярского края Дмитрию Гушану и его сообщнику вынесен приговор за незаконную покупку взлетно-посадочной полосы. Об этом в понедельник, 10 декабря, в прокуратуре Красноярского края сообщили газете «Коммерсантъ». Как установило следствие, в период с 2013 по 2014 год Гушану и директор компании «Ориентир» незаконно приобрели право на железобетонную взлетно-посадочную полосу бывшего аэропорта Ирбей и земельный участок под ним. Ущерб, нанесенный администрации Ирбейского района, составил 20,9 миллиона рублей. В отношении сообщников было возбуждено уголовное дело по части 4 статьи 159 УК РФ («Мошенничество в особо крупном размере»). Канский городской суд Красноярского края признал подсудимых виновными и с учетом смягчающих обстоятельств  приговорил каждого к трем годам лишения свободы условно. Подсудимые возместили ущерб на сумму почти 300 тысяч рублей. 7 сентября 2018 года сообщалось, что Мосгорсуд сн

Вывод: В принципе, если взять идею о том, что первая статья была о преступлении (генцид - это нехорошо), а так же то, что речь идет о международной политике, то можно считать, что моделька справилась успешно